In [7]:
from py2neo import Graph,Node,Relationship
import numpy as np
import json
import os

In [8]:
graph = Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="doramon401"
)

In [9]:
graph.delete_all()

In [10]:
def load_one_file(fn):
    #load the json file
#     f = file(fn)
    with open(fn) as f:
        test = json.load(f)
        
    #save the article/doc. node
    article = Node('ARTICLE',name = test['TITLE'], ID = test['ID'])
    #save the author node
    author = Node("PERSON",name = test['AUTHOR'])
    #graph.merge(): carried out by comparing that node with a potential remote equivalent 
    #on the basis of a label and property value. If no remote match is found, a new node is created. 
    graph.merge(article|author)
    graph.merge(Relationship(author,'WRITES',article))
    #save the location nodes
    for i in np.arange(len(test['LOCATION'])):
        location = Node("LOCATION",name = test['LOCATION'][i])
        graph.merge(location)
        graph.merge(Relationship(article,'MENTIONS',location))
    #save the organization nodes
    for j in np.arange(len(test['ORGANIZATION'])):
        organization = Node("ORGANIZATION",name = test['ORGANIZATION'][j])
        graph.merge(organization)
        graph.merge(Relationship(article,'MENTIONS',organization))
    #save the person nodes
    #person who is mentioned and the author will be saved in the same label "PERSON"
    #because author may be mentioned in other doc.
    for k in np.arange(len(test['PERSON'])):
        person = Node("PERSON",name = test['PERSON'][k])
        graph.merge(person)
        graph.merge(Relationship(article,'MENTIONS',person))

In [11]:
def getFileList( p ):
    p = str( p )
    if p=="":
        return [ ]
#     For windows
#     p = p.replace( "/","\\")
#     if p[ -1] != "\\":
#         p = p+"\\"
    a = os.listdir( p )
    b = [ x   for x in a if os.path.isfile( p + x ) ]
    return b

In [12]:
fileList = getFileList("json/")

In [13]:
def load_files(folderPath):
    fileList = getFileList(folderPath)
    for v in np.arange(len(fileList)):
    #for v in np.arange(9):
        load_one_file(folderPath+fileList[v])

In [14]:
load_files("json/")